In [67]:
from __future__ import print_function 
from __future__ import division 
import os 
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import numpy as np
from Cleaner import api as p
import warnings 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [18]:
# You may have problems with preprocessor
# to Install
# pip install git+git://github.com/s/preprocessor.git@master

In [5]:
datadir = os.path.realpath('../data') 
with pd.HDFStore(os.path.join(datadir, 'results.h5')) as store: 
    tweets = store.results 

In [7]:
tweets.head()

,lat,lng,text,timeStamp,user_id,mb_cluster,db_cluster,cluster,hdb_cluster,final_cluster
id,,,,,,,,,,
377652254096228352,37.446100,-121.883557,@Tanner_Cortez hey checkout the website: http:...,2013-09-11 04:38:08,224874450,44,-1.0,NaN,-1.0,NaN
377652262325456897,37.356131,-121.842867,i laugh a lot with that line,2013-09-11 04:38:10,54351774,1,-1.0,NaN,-1.0,NaN
377652264682655744,37.364664,-122.009629,sons of anarchy is back on woop woop,2013-09-11 04:38:11,343219606,34,-1.0,NaN,-1.0,NaN
377652271116722176,37.382600,-121.995000,Drinking a Fresh Squeezed IPA by @deschutesbee...,2013-09-11 04:38:12,1569395935,34,-1.0,NaN,-1.0,NaN
377652275147444224,37.756149,-122.152813,I have 8 am classes this quarter ... I need to...,2013-09-11 04:38:13,399164195,31,0.0,31.0,0.0,31.0


In [8]:
tweets.columns

Index(['lat', 'lng', 'text', 'timeStamp', 'user_id', 'mb_cluster',
       'db_cluster', 'cluster', 'hdb_cluster', 'final_cluster'],
      dtype='object')

# Handle Emoticons and Emojis

In [9]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

In [10]:
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

In [11]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [12]:
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [68]:
def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    tweet = emoji_pattern.sub(r'', tweet)
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
    for w in word_tokens:
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)

In [70]:
cleaned_text = []
for singletext in tweets["text"]:
    clean_text = p.clean(singletext)
    filtered_tweet=clean_tweets(clean_text)
    cleaned_text.append(filtered_tweet)

print("Cleaning Successful")
print("Sample: ", cleaned_text[:5])
# p.clean('Preprocessor is #awesome 👍 https://github.com/s/preprocessor')

Cleaning Successful
Sample:  ['hey checkout website', 'laugh lot line', 'sons anarchy back woop woop', "Drinking Fresh Squeezed IPA St. John 's Bar amp Grill —", 'I classes quarter ... I need get sleep thing together']


In [75]:
tweets["Cleaned_Initial"] = cleaned_text

# Vocabulary and Word Counts for IDF

In [71]:
from sklearn.feature_extraction.text import CountVectorizer

In [80]:
stop_set = set(stopwords.words('english'))
frozenset(stop_set)
print("")

In [81]:
docs = tweets["Cleaned_Initial"].tolist()

In [107]:
tweets['separate_words'] = tweets.text.str.strip().str.split('[\W_]+')

In [95]:
cv = CountVectorizer(docs,max_df=0.85)

In [96]:
word_count_vector = cv.fit_transform(docs)

In [97]:
new_vocab = {}
for key,value in cv.vocabulary_.items():
    if(len(key) < 4):
        pass
    else:
        new_vocab[key] = value

In [98]:
vocab_df = pd.DataFrame(list(new_vocab.items()))

In [99]:
vocab_df.head()

,0,1
0,checkout,25290
1,website,147619
2,laugh,78212
3,line,80123
4,sons,126833


In [119]:
vocab_df.to_csv("../data/results/Vocab_Frame.csv")

# TfidfTransformer to Compute Inverse Document Frequency (IDF)

In [104]:
from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [109]:
# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
df_idf.sort_values(by=['idf_weights'])

,idf_weights
ca,3.994374
like,4.102876
lol,4.275962
san,4.409975
the,4.441400
...,...
krustiee,13.909247
krust,13.909247
krusher,13.909247
krys,13.909247


In [110]:
count_vector=cv.transform(docs) 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [113]:
feature_names = cv.get_feature_names()

In [114]:
first_document_vector=tf_idf_vector[0]

In [116]:
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
checkout,0.692689
website,0.584278
hey,0.422850
seacliff,0.000000
seabitches,0.000000
...,...
halfs,0.000000
halftime,0.000000
halftimes,0.000000
halfway,0.000000


In [118]:
df.to_csv("../data/results/TFIDF_SparseMatrix.csv")

In [120]:
tweets.columns

Index(['lat', 'lng', 'text', 'timeStamp', 'user_id', 'mb_cluster',
       'db_cluster', 'cluster', 'hdb_cluster', 'final_cluster',
       'Cleaned_Initial', 'separate_words'],
      dtype='object')

In [121]:
with pd.HDFStore(os.path.join(datadir, 'results.h5'), mode='w') as results: 
    results['results'] = tweets

c:\users\faizan\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3326: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->['text', 'Cleaned_Initial', 'separate_words']]

  exec(code_obj, self.user_global_ns, self.user_ns)
